# 05 - GridSearchCV - CatBoost

#### Imports

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="white")

#### Constants

In [2]:
n_components = 1000

In [3]:
models_folder = "models/"
train_data_fn = models_folder+'train_data.pkl'
target_fn = models_folder+'target.pkl'
test_data_fn = models_folder+'test_data.pkl'

weight_multiplier_fn = models_folder+"weight_multiplier.pkl"

#### Functions

In [4]:
import os.path
from sklearn.externals import joblib

def Load(filename):
    if os.path.isfile(filename):
        return joblib.load(filename)
    
def Save(obj, filename):
    joblib.dump(obj, filename)

# Loading data

In [5]:
data = Load(train_data_fn)
target = Load(target_fn)

In [6]:
weight_multiplier = Load(weight_multiplier_fn)

## Splitting dataset

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_validation, Y_train, Y_validation = train_test_split(data, target.ravel(), train_size=0.8, random_state=42)

/home/aavdeev/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


# CatBoost Classifier

In [38]:
tuned_parameters = {
     'iterations':[500,1000,5000],
     'depth':[3,6,10],
     'l2_leaf_reg':[2,3,10,20],
     'model_size_reg':[0.5,2,10],
     'rsm':[0.1,0.5,1],
     'loss_function':['Logloss'],
     'border_count':[100,128,200],
     'leaf_estimation_iterations':[1,2,10],
     'leaf_estimation_method':['Newton','Gradient'],
     'random_seed':[42],
     'metric_period':[1,10,100],
     'one_hot_max_size':[1,2,10],
     'random_strength':[2,3],
     'eval_metric':['AUC'],
     'bagging_temperature':[0.2,0.6,1],
     'fold_len_multiplier':[2,5,10],
     'task_type':['CPU'],
#      'bootstrap_type':['Bayesian','Bernoulli'],
#      'subsample':[0.66,0.3,0.9],
     'class_weights':[[1, 1],[1,1/weight_multiplier],[1,weight_multiplier]]
#      'scale_pos_weight':[None,1/weight_multiplier,weight_multiplier]
}

In [39]:
%%time
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from catboost import CatBoostClassifier

clf = RandomizedSearchCV(CatBoostClassifier(),
                   tuned_parameters,
                   cv=4,
                   n_iter=5,
                   n_jobs=4,
                   scoring='roc_auc',
                   random_state=42,
                   verbose=2)

CPU times: user 226 µs, sys: 15 µs, total: 241 µs
Wall time: 247 µs


In [40]:
%%time
clf.fit(X_train, Y_train)

Fitting 4 folds for each of 5 candidates, totalling 20 fits
[CV] task_type=CPU, rsm=0.5, random_strength=3, random_seed=42, one_hot_max_size=10, model_size_reg=2, metric_period=1, loss_function=Logloss, leaf_estimation_method=Newton, leaf_estimation_iterations=1, l2_leaf_reg=20, iterations=500, fold_len_multiplier=5, eval_metric=AUC, depth=3, class_weights=[1, 1], border_count=128, bagging_temperature=1 
[CV] task_type=CPU, rsm=0.5, random_strength=3, random_seed=42, one_hot_max_size=10, model_size_reg=2, metric_period=1, loss_function=Logloss, leaf_estimation_method=Newton, leaf_estimation_iterations=1, l2_leaf_reg=20, iterations=500, fold_len_multiplier=5, eval_metric=AUC, depth=3, class_weights=[1, 1], border_count=128, bagging_temperature=1 
[CV] task_type=CPU, rsm=0.5, random_strength=3, random_seed=42, one_hot_max_size=10, model_size_reg=2, metric_period=1, loss_function=Logloss, leaf_estimation_method=Newton, leaf_estimation_iterations=1, l2_leaf_reg=20, iterations=500, fold_len

1:	learn: 0.5884988	total: 1.9s	remaining: 7m 54s
79:	learn: 0.6240909	total: 59.6s	remaining: 5m 13s
51:	learn: 0.6169500	total: 45s	remaining: 6m 27s
80:	learn: 0.6245640	total: 1m	remaining: 5m 12s
2:	learn: 0.5997397	total: 2.84s	remaining: 7m 51s
52:	learn: 0.6170543	total: 45.8s	remaining: 6m 26s
3:	learn: 0.5986921	total: 3.7s	remaining: 7m 38s
81:	learn: 0.6245665	total: 1m 1s	remaining: 5m 13s
53:	learn: 0.6174335	total: 46.8s	remaining: 6m 26s
4:	learn: 0.5988286	total: 4.55s	remaining: 7m 31s
82:	learn: 0.6246935	total: 1m 2s	remaining: 5m 12s
54:	learn: 0.6181632	total: 47.5s	remaining: 6m 24s
83:	learn: 0.6248162	total: 1m 3s	remaining: 5m 12s
5:	learn: 0.5984383	total: 5.63s	remaining: 7m 43s
55:	learn: 0.6185553	total: 48.5s	remaining: 6m 24s
84:	learn: 0.6249574	total: 1m 3s	remaining: 5m 11s
6:	learn: 0.6005675	total: 6.59s	remaining: 7m 44s
56:	learn: 0.6184485	total: 49.3s	remaining: 6m 23s
85:	learn: 0.6248182	total: 1m 4s	remaining: 5m 11s
7:	learn: 0.6015235	total

93:	learn: 0.6240168	total: 1m 24s	remaining: 6m 3s
44:	learn: 0.6157288	total: 41.4s	remaining: 6m 59s
124:	learn: 0.6294627	total: 1m 39s	remaining: 4m 58s
29:	learn: 0.6125951	total: 27.2s	remaining: 7m 5s
45:	learn: 0.6159629	total: 42.3s	remaining: 6m 57s
94:	learn: 0.6241950	total: 1m 25s	remaining: 6m 3s
125:	learn: 0.6292611	total: 1m 40s	remaining: 4m 57s
30:	learn: 0.6125428	total: 28.1s	remaining: 7m 4s
46:	learn: 0.6163683	total: 43.2s	remaining: 6m 56s
95:	learn: 0.6242463	total: 1m 26s	remaining: 6m 2s
126:	learn: 0.6293643	total: 1m 41s	remaining: 4m 57s
31:	learn: 0.6135431	total: 29s	remaining: 7m 4s
47:	learn: 0.6160867	total: 44.1s	remaining: 6m 55s
96:	learn: 0.6245082	total: 1m 27s	remaining: 6m 1s
127:	learn: 0.6294121	total: 1m 42s	remaining: 4m 57s
32:	learn: 0.6142004	total: 29.9s	remaining: 7m 2s
48:	learn: 0.6163087	total: 45s	remaining: 6m 54s
97:	learn: 0.6249700	total: 1m 27s	remaining: 6m
33:	learn: 0.6143240	total: 30.7s	remaining: 7m
128:	learn: 0.62974

162:	learn: 0.6337541	total: 2m 15s	remaining: 4m 40s
131:	learn: 0.6290087	total: 2m 1s	remaining: 5m 38s
69:	learn: 0.6234461	total: 1m 3s	remaining: 6m 32s
84:	learn: 0.6213468	total: 1m 18s	remaining: 6m 24s
163:	learn: 0.6338324	total: 2m 16s	remaining: 4m 39s
132:	learn: 0.6289574	total: 2m 2s	remaining: 5m 38s
85:	learn: 0.6214174	total: 1m 19s	remaining: 6m 24s
70:	learn: 0.6237847	total: 1m 4s	remaining: 6m 32s
164:	learn: 0.6338032	total: 2m 17s	remaining: 4m 38s
133:	learn: 0.6291114	total: 2m 3s	remaining: 5m 37s
86:	learn: 0.6219539	total: 1m 20s	remaining: 6m 23s
165:	learn: 0.6340116	total: 2m 18s	remaining: 4m 38s
71:	learn: 0.6236339	total: 1m 5s	remaining: 6m 31s
134:	learn: 0.6292258	total: 2m 4s	remaining: 5m 36s
166:	learn: 0.6342210	total: 2m 19s	remaining: 4m 37s
87:	learn: 0.6219986	total: 1m 21s	remaining: 6m 23s
72:	learn: 0.6236874	total: 1m 7s	remaining: 6m 32s
135:	learn: 0.6290647	total: 2m 5s	remaining: 5m 35s
167:	learn: 0.6345198	total: 2m 20s	remaining

201:	learn: 0.6380125	total: 2m 52s	remaining: 4m 14s
122:	learn: 0.6271291	total: 1m 54s	remaining: 5m 51s
170:	learn: 0.6332668	total: 2m 37s	remaining: 5m 3s
107:	learn: 0.6290149	total: 1m 40s	remaining: 6m 2s
202:	learn: 0.6380721	total: 2m 53s	remaining: 4m 13s
171:	learn: 0.6333073	total: 2m 38s	remaining: 5m 2s
123:	learn: 0.6273892	total: 1m 55s	remaining: 5m 50s
108:	learn: 0.6292340	total: 1m 40s	remaining: 6m 1s
203:	learn: 0.6381654	total: 2m 54s	remaining: 4m 12s
172:	learn: 0.6335590	total: 2m 39s	remaining: 5m 1s
109:	learn: 0.6295847	total: 1m 41s	remaining: 6m
124:	learn: 0.6273256	total: 1m 56s	remaining: 5m 50s
204:	learn: 0.6382356	total: 2m 55s	remaining: 4m 12s
173:	learn: 0.6337929	total: 2m 40s	remaining: 5m
125:	learn: 0.6272475	total: 1m 57s	remaining: 5m 49s
110:	learn: 0.6294149	total: 1m 42s	remaining: 6m
205:	learn: 0.6384332	total: 2m 56s	remaining: 4m 11s
126:	learn: 0.6273584	total: 1m 58s	remaining: 5m 48s
174:	learn: 0.6338098	total: 2m 41s	remaining

...

468:	learn: 0.7032029	total: 1m 44s	remaining: 1m 58s
469:	learn: 0.7033522	total: 1m 45s	remaining: 1m 58s
470:	learn: 0.7034553	total: 1m 45s	remaining: 1m 58s
471:	learn: 0.7036003	total: 1m 45s	remaining: 1m 58s
472:	learn: 0.7037238	total: 1m 45s	remaining: 1m 57s
473:	learn: 0.7038234	total: 1m 46s	remaining: 1m 57s
474:	learn: 0.7039309	total: 1m 46s	remaining: 1m 57s
475:	learn: 0.7040351	total: 1m 46s	remaining: 1m 57s
476:	learn: 0.7041464	total: 1m 46s	remaining: 1m 57s
477:	learn: 0.7042710	total: 1m 46s	remaining: 1m 56s
478:	learn: 0.7043350	total: 1m 47s	remaining: 1m 56s
479:	learn: 0.7044319	total: 1m 47s	remaining: 1m 56s
480:	learn: 0.7045445	total: 1m 47s	remaining: 1m 56s
481:	learn: 0.7046826	total: 1m 47s	remaining: 1m 55s
482:	learn: 0.7048203	total: 1m 48s	remaining: 1m 55s
483:	learn: 0.7049444	total: 1m 48s	remaining: 1m 55s
484:	learn: 0.7050602	total: 1m 48s	remaining: 1m 55s
485:	learn: 0.7051644	total: 1m 48s	remaining: 1m 54s
486:	learn: 0.7052436	total:

621:	learn: 0.7202399	total: 2m 17s	remaining: 1m 23s
622:	learn: 0.7202775	total: 2m 17s	remaining: 1m 23s
623:	learn: 0.7203926	total: 2m 18s	remaining: 1m 23s
624:	learn: 0.7204966	total: 2m 18s	remaining: 1m 23s
625:	learn: 0.7206288	total: 2m 18s	remaining: 1m 22s
626:	learn: 0.7207330	total: 2m 18s	remaining: 1m 22s
627:	learn: 0.7208509	total: 2m 19s	remaining: 1m 22s
628:	learn: 0.7209736	total: 2m 19s	remaining: 1m 22s
629:	learn: 0.7210714	total: 2m 19s	remaining: 1m 21s
630:	learn: 0.7212220	total: 2m 19s	remaining: 1m 21s
631:	learn: 0.7213521	total: 2m 19s	remaining: 1m 21s
632:	learn: 0.7214934	total: 2m 20s	remaining: 1m 21s
633:	learn: 0.7216071	total: 2m 20s	remaining: 1m 21s
634:	learn: 0.7217111	total: 2m 20s	remaining: 1m 20s
635:	learn: 0.7217898	total: 2m 20s	remaining: 1m 20s
636:	learn: 0.7219084	total: 2m 20s	remaining: 1m 20s
637:	learn: 0.7220105	total: 2m 21s	remaining: 1m 20s
638:	learn: 0.7221260	total: 2m 21s	remaining: 1m 19s
639:	learn: 0.7221764	total:

775:	learn: 0.7357649	total: 2m 50s	remaining: 49.4s
776:	learn: 0.7358659	total: 2m 51s	remaining: 49.1s
777:	learn: 0.7359513	total: 2m 51s	remaining: 48.9s
778:	learn: 0.7360680	total: 2m 51s	remaining: 48.7s
779:	learn: 0.7361823	total: 2m 51s	remaining: 48.5s
780:	learn: 0.7362751	total: 2m 52s	remaining: 48.2s
781:	learn: 0.7363857	total: 2m 52s	remaining: 48s
782:	learn: 0.7364855	total: 2m 52s	remaining: 47.8s
783:	learn: 0.7365746	total: 2m 52s	remaining: 47.6s
784:	learn: 0.7366213	total: 2m 52s	remaining: 47.4s
785:	learn: 0.7367078	total: 2m 53s	remaining: 47.1s
786:	learn: 0.7368146	total: 2m 53s	remaining: 46.9s
787:	learn: 0.7369162	total: 2m 53s	remaining: 46.7s
788:	learn: 0.7370117	total: 2m 53s	remaining: 46.5s
789:	learn: 0.7371309	total: 2m 53s	remaining: 46.2s
790:	learn: 0.7372421	total: 2m 54s	remaining: 46s
791:	learn: 0.7373290	total: 2m 54s	remaining: 45.8s
792:	learn: 0.7374409	total: 2m 54s	remaining: 45.6s
793:	learn: 0.7375275	total: 2m 54s	remaining: 45.

932:	learn: 0.7500793	total: 3m 24s	remaining: 14.7s
933:	learn: 0.7501524	total: 3m 24s	remaining: 14.5s
934:	learn: 0.7502568	total: 3m 25s	remaining: 14.3s
935:	learn: 0.7503575	total: 3m 25s	remaining: 14s
936:	learn: 0.7504850	total: 3m 25s	remaining: 13.8s
937:	learn: 0.7505818	total: 3m 25s	remaining: 13.6s
938:	learn: 0.7506769	total: 3m 26s	remaining: 13.4s
939:	learn: 0.7506904	total: 3m 26s	remaining: 13.2s
940:	learn: 0.7507987	total: 3m 26s	remaining: 12.9s
941:	learn: 0.7509040	total: 3m 26s	remaining: 12.7s
942:	learn: 0.7510130	total: 3m 26s	remaining: 12.5s
943:	learn: 0.7511121	total: 3m 27s	remaining: 12.3s
944:	learn: 0.7511693	total: 3m 27s	remaining: 12.1s
945:	learn: 0.7512174	total: 3m 27s	remaining: 11.8s
946:	learn: 0.7513444	total: 3m 27s	remaining: 11.6s
947:	learn: 0.7514076	total: 3m 27s	remaining: 11.4s
948:	learn: 0.7514535	total: 3m 28s	remaining: 11.2s
949:	learn: 0.7515466	total: 3m 28s	remaining: 11s
950:	learn: 0.7516324	total: 3m 28s	remaining: 10.

RandomizedSearchCV(cv=4, error_score='raise',
          estimator=<catboost.core.CatBoostClassifier object at 0x7fbe25c1b240>,
          fit_params=None, iid=True, n_iter=5, n_jobs=4,
          param_distributions={'iterations': [500, 1000, 5000], 'depth': [3, 6, 10], 'l2_leaf_reg': [2, 3, 10, 20], 'model_size_reg': [0.5, 2, 10], 'rsm': [0.1, 0.5, 1], 'loss_function': ['Logloss'], 'border_count': [100, 128, 200], 'leaf_estimation_iterations': [1, 2, 10], 'leaf_estimation_method': ['Newton'... 'task_type': ['CPU'], 'class_weights': [[1, 1], [1, 0.05276699578395344], [1, 18.951239977624464]]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring='roc_auc', verbose=2)

In [41]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [42]:
print("RandomizedSearchCV")
report(clf.cv_results_)

RandomizedSearchCV
Model with rank: 1
Mean validation score: 0.638 (std: 0.005)
Parameters: {'task_type': 'CPU', 'rsm': 0.1, 'random_strength': 3, 'random_seed': 42, 'one_hot_max_size': 10, 'model_size_reg': 10, 'metric_period': 1, 'loss_function': 'Logloss', 'leaf_estimation_method': 'Newton', 'leaf_estimation_iterations': 1, 'l2_leaf_reg': 2, 'iterations': 1000, 'fold_len_multiplier': 2, 'eval_metric': 'AUC', 'depth': 6, 'class_weights': [1, 18.951239977624464], 'border_count': 200, 'bagging_temperature': 1}

Model with rank: 2
Mean validation score: 0.637 (std: 0.004)
Parameters: {'task_type': 'CPU', 'rsm': 0.5, 'random_strength': 3, 'random_seed': 42, 'one_hot_max_size': 10, 'model_size_reg': 2, 'metric_period': 1, 'loss_function': 'Logloss', 'leaf_estimation_method': 'Newton', 'leaf_estimation_iterations': 1, 'l2_leaf_reg': 20, 'iterations': 500, 'fold_len_multiplier': 5, 'eval_metric': 'AUC', 'depth': 3, 'class_weights': [1, 1], 'border_count': 128, 'bagging_temperature': 1}

Mod

In [43]:
params = clf.best_params_
# params['n_jobs']=-1
# params['verbose']=2

In [44]:
model = clf.best_estimator_

In [ ]:
from catboost import Pool

train_pool = Pool(pd.DataFrame(X_train), pd.DataFrame(Y_train))

In [ ]:
%%time 
from catboost import CatBoostClassifier

model = CatBoostClassifier(    
    od_type = 'Iter',
    od_wait = 150,
    use_best_model = True,
    loss_function='Logloss',
#     loss_function='CrossEntropy',
    eval_metric='AUC',
    train_dir = 'mlbootcamp_catboost_train_dir', 
    random_state=42,
#     class_weights=[1, weight_multiplier],
    scale_pos_weight=weight_multiplier,
#     task_type='CPU'
    task_type='CPU'
    
)

In [ ]:
model.fit(
    train_pool,
    eval_set=(pd.DataFrame(X_validation), pd.DataFrame(Y_validation)),
    verbose=True,
    plot=True
);

In [45]:
from sklearn.metrics import roc_auc_score

predicted = model.predict(X_validation)
print("ROC AUC score:",roc_auc_score(Y_validation, predicted))

ROC AUC score: 0.5959288891669104


In [46]:
model.save_model('catboost_model.cbm')

In [ ]:
# svd = Load(svd_fn)

In [ ]:
# best_features = [features[i] for i in svd.components_[0].argsort()[::-1]]
# feature_importance = model.get_feature_importance(train_pool)
# imp_data = pd.DataFrame({'importance' : feature_importance}, index = best_features[:n_components])
# imp_data.to_pickle('mlbootcamp_model_importance.pkl')
# imp_data.plot(kind='bar', color='red')

# Test Data

In [ ]:
# model = model.load_model('mlbootcamp_model.cbm')

In [47]:
data = Load(test_data_fn)

In [48]:
Y_test = model.predict(data)

## Saving test predictions

In [49]:
predictions = pd.DataFrame(Y_test)
predictions.to_csv("solution_catboost.csv",header=None, index=None)